# PROMETEO_v1.0

## Paquetería

In [ ]:
import ast
import numpy as np
import os
import pandas as pd
import textwrap
import tiktoken
import umap.umap_ as umap
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
from typing import Optional

## Funciones

In [ ]:
def num_tokens_from_string(string: str) -> int:
    """Cuenta el número de tokens en el documento
    proporcionado."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def reduce_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """Esta función no la comprendo. Estudie
    para entenderla."""
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

def get_optimal_clusters(embeddings: np.ndarray, max_clusters: int = 50, random_state: int = 1234):
    """Obtiene el número óptimo de clústers."""
    max_clusters = min(max_clusters, len(embeddings))
    bics = [GaussianMixture(n_components=n, random_state=random_state).fit(embeddings).bic(embeddings)
            for n in range(1, max_clusters)]
    return np.argmin(bics) + 1

def gmm_clustering(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """Clusteriza con el método GMM."""
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state).fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters

def format_cluster_texts(df):
    """Agrupa los textos de cada clúster el listas."""
    clustered_texts = {}
    for cluster in df['Cluster'].unique():
        cluster_texts = df[df['Cluster'] == cluster]['Texto'].tolist()
        clustered_texts[cluster] = " --- ".join(cluster_texts)
    return clustered_texts

def wrap_text_preserve_newlines(text, width=80):
    """Formato para respuestas."""
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    """Generador de referencias."""
    print(wrap_text_preserve_newlines(llm_response['answer']))
    print('\nReferencias:')
    for contexto in llm_response["context"][:5]:
        print(contexto)
    print('\n\n')

## Parámetros

In [ ]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings(
)

detailed_turbo_llm = turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4o-mini'
)

template = """Tu tarea es generar un resumen extremadamente detallado del siguiente
texto: {text} """

prompt = PromptTemplate.from_template(template)
chain = prompt | detailed_turbo_llm | StrOutputParser()

turbo_llm = ChatOpenAI(
    temperature=0.5,
    model_name='gpt-4o-mini'
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
)

## ¿Quieres crear o cargar un tema de conversación?

In [ ]:
user_input = input("¿Crear (1) o cargar (2) un tema de conversación?: ")

if user_input.lower() == "1":
    print(f'Elegiste crear de cero un tema de conversación.\n')

    # Asegúrate de que haya PDFs en la carpeta 'docs'
    documents = DirectoryLoader('./docs/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
    # Tratameinto de caracteres indeseados
    for d in documents:
        d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')

    docs = text_splitter.split_documents(documents)
    texts = [doc.page_content for doc in docs]

    d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
    d_reversed = list(reversed(d_sorted))
    concatenated_content = "\n\n\n --- \n\n\n".join(
        [doc.page_content for doc in d_reversed]
    )
    print(
        "Número de tokens en el documento proporcionado: %s"
        % num_tokens_from_string(concatenated_content)
    )

    global_embeddings = [embeddings.embed_query(txt) for txt in texts]

    topic_name = input('¿Cómo se llama el tema de conversación?')

    embed_name = topic_name + '_emb' + '.txt'
    with open(rf'./{embed_name}', 'w') as f:
        for i in global_embeddings:
            f.write("%s\n" % i)
    print(f'\nEstás usando el tema de conversación: {embed_name}')

    dim = 2
    global_embeddings_reduced = reduce_cluster_embeddings(global_embeddings, dim)
    labels, _ = gmm_clustering(global_embeddings_reduced, threshold=0.5)
    simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]

    df = pd.DataFrame({
        'Texto': texts,
        'Embedding': list(global_embeddings_reduced),
        'Cluster': simple_labels
    })

    clustered_texts = format_cluster_texts(df)
    summaries = {}
    for cluster, text in clustered_texts.items():
        summary = chain.invoke({"text": text})
        summaries[cluster] = summary
    embedded_summaries = [embeddings.embed_query(summary) for summary in summaries.values()]
    embedded_summaries_np = np.array(embedded_summaries)
    labels, _ = gmm_clustering(embedded_summaries_np, threshold=0.5)
    simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]
    clustered_summaries = {}
    for i, label in enumerate(simple_labels):
        if label not in clustered_summaries:
            clustered_summaries[label] = []
        clustered_summaries[label].append(list(summaries.values())[i])
    final_summaries = {}
    for cluster, texts in clustered_summaries.items():
        combined_text = ' '.join(texts)
        summary = chain.invoke({"text": combined_text})
        final_summaries[cluster] = summary
    texts_from_df = df['Texto'].tolist()
    texts_from_clustered_texts = list(clustered_texts.values())
    texts_from_final_summaries = list(final_summaries.values())

    combined_texts = texts_from_df + texts_from_clustered_texts + texts_from_final_summaries

    file_name = topic_name + '.txt'
    with open(file_name, 'w', encoding='utf-8') as f:
        for t in combined_texts:
            f.write("%s\n" % t)

    with open(file_name, 'r', encoding='utf-8') as f:
        content = f.read()

    textos = text_splitter.split_text(content)

    persist_directory = topic_name + '_kb'
    vectorstore = Chroma.from_texts(texts=textos,
                                    embedding=embeddings,
                                    persist_directory=persist_directory)
    vectorstore.persist()
    vectorstore = None
    os.system(f'zip -r db.zip ./{persist_directory}')

    vectorstore = Chroma(persist_directory=persist_directory,
                         embedding_function=embeddings)

    def adjust_final_number(string: str, max_threshold: int, initial_number: int) -> int:
        final_number = initial_number
        while final_number < max_threshold:
            retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
            docs = retriever.invoke(string)
            text = "".join([doc.page_content for doc in docs])
            if num_tokens_from_string(text) < max_threshold:
                final_number += 1
            else:
                break
        return final_number

    final_number = adjust_final_number("¿Cuál es el tema principal del documento?", 10000, 4)
    print(f'\nK final es: {final_number}')
    retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
    
elif user_input.lower() == "2":
    print('Elegiste cargar un tema de conversación ya creado.\n')
    global_embeddings = []

    topic_name = input('¿Cómo se llama el tema de conversación?')

    embed_name = topic_name + '_emb' + '.txt'
    print(f'Estás usando el tema de conversación: {embed_name}\n')
    with open(rf'./{embed_name}', 'r') as f:
        for i in f:
            x = ast.literal_eval(i.strip())  # Convertir la cadena a lista de números
            global_embeddings.append(x)

    global_embeddings = np.array(global_embeddings, dtype=float)

    file_name = topic_name + '.txt'
    with open(file_name, 'r', encoding='utf-8') as f:
        content = f.read()

    textos = text_splitter.split_text(content)

    persist_directory = topic_name + '_kb'
    vectorstore = Chroma(persist_directory=persist_directory, 
                    embedding_function=embeddings)

    def adjust_final_number(string: str, max_threshold: int, initial_number: int) -> int:
        final_number = initial_number
        while final_number < max_threshold:
            retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
            docs = retriever.invoke(string)
            text = "".join([doc.page_content for doc in docs])
            if num_tokens_from_string(text) < max_threshold:
                final_number += 1
            else:
                break
        return final_number

    final_number = adjust_final_number("¿Cuál es el tema principal del documento?", 10000, 4)
    print(f'K final es: {final_number}')
    retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
    
elif user_input != "1" and user_input != "2":
    print('No seleccionaste ningún tema de conversación.\n')

# Se personaliza el LLM #

print('Prometeo (1): Revisor bibliográfico.\nYves (2): Diseñador de cartillas.')
print('\n\n')

template_select = input('Prometeo (1); Yves (2): ')

if template_select == "1":

    name = 'Prometeo'

    template = """
    Eres Prometeo, un asistente personal especializado en revisión biliográfica que habla Español.

    Tu tarea consiste en:

    1. Leer detalladamente la información proporcionada en documentos que generalmente son
    artículos científicos en formato PDF.

    2. Proporcionar respuestas extremadamente detalladas a cualquier tipo de pregunta relacionada 
    con el siguiente contexto: {context}.

    SIEMPRE debes responder en Español, con excepción de nombres propios.

    NUNCA hables específicamente del contexto proporcionado.

    NUNCA hables de ti a menos de que la pregunta lo requiera.

    Teniendo en cuenta TODO lo anterior, responde la siguiente pregunta: {question}
    """

elif template_select == "2":

    name = 'Yves'

    template = """
    Eres Yves, un asistente personal que habla Español, especializado en el diseño y desarrollo de cartillas 
    pedagógicas para equipos de investigación universitarios, basada en el tema de "Lean Startup" como metodología 
    para emprender.
    
    Tu tarea consiste en:

    1. Leer detalladamente la información proporcionada en documentos que generalmente son
    artículos científicos en formato PDF.
    
    2. Asegurando que el contenido sea claro, accesible y de alta calidad, debes ayudar a los equipos 
    a recopilar y organizar información relevante del siguiente contexto: {context}.

    SIEMPRE debes responder en Español, con excepción de nombres propios.

    NUNCA hables específicamente del contexto proporcionado.

    NUNCA hables de ti a menos de que la pregunta lo requiera.

    Teniendo en cuenta TODO lo anterior, apoya al equipo con la siguiente petición: {question}
    """

elif template_select != "1" and template_select != "2":
    print('No seleccionaste ningún asistente.\n')

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prometeo_prompt
    | turbo_llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain
)

In [ ]:
def ask_questions():
    while True:
        query = input(f"¡Hola, soy {name}! ¿En qué te puedo servir?: ")
        print(query)
        llm_response = rag_chain_with_source.invoke(query)
        process_llm_response(llm_response)
        print('\n\n')
        
        another_question = input("¿Te puedo servir en algo más? (s/n): ").strip().lower()
        if another_question != 's':
            break

In [ ]:
# Prompt: Genera un mapa mental con los puntos claves del documento.

if __name__ == "__main__":
    ask_questions()

In [ ]:
# prometeo_preguntas_meta = ['¿En qué año fue redactado el artículo?',
#                   '¿En qué revista o medio fue publicado el artículo?',
#                   '¿En qué país fue redactado el artículo?',
#                   '¿Cuál es el título original y traducido del artículo?',
#                   '¿Quién o quiénes son los autores del artículo?',
#                   '¿Cuáles son las palabras clave de investigación mencionadas en el documento y traducidas al español, del artículo?'
#                   '¿Cuál es la idea principal de investigación?',
#                   '¿Cuál es el objetivo u objetivos de la investigación?',
#                   '¿Cuál es la metodología usada en la investigación?',
#                   '¿Cuáles fueron los resultados más importantes de la investigación?',
#                   '¿Los resultados cumplen los objetivos de investigación?',
#                   '¿Cómo puede aportar esta investigación al diseño, desarrollo y puesta en marcha de un laboratorio de investigación para el programa de Administración de Empresas en una universidad?'
#                   ]

# for p in prometeo_preguntas_meta:
#     query = p
#     print(query)
#     llm_response = rag_chain_with_source.invoke(query)
#     process_llm_response(llm_response)
#     print('\n')

In [ ]:
# # Demo
# query = input("Hazme una pregunta: ")
# print(query)
# llm_response = rag_chain_with_source.invoke(query)
# process_llm_response(llm_response)

# Hacer historial y guardar conversaciones para medir efectividad.